In [8]:

import gradio as gr
import os
import cv2
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import io
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from openai import AzureOpenAI

# Custom Vision 설정
CUSTOM_VISION_ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
CUSTOM_VISION_PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
CUSTOM_VISION_PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
CUSTOM_VISION_PUBLISHED_NAME = "Iteration8"

# Custom Vision 클라이언트 초기화
customvision_credentials = ApiKeyCredentials(in_headers={"Prediction-key": CUSTOM_VISION_PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=CUSTOM_VISION_ENDPOINT, credentials=customvision_credentials)

# 자음/모음 분리 함수
class HangulSplitter:
    def __init__(self):
        self.initial_consonants = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
        self.medial_vowels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
        self.final_consonants = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    def split_hangul(self, word): # 단어 초성 중성 종성 분리
        separated = []
        for char in word:
            if '가' <= char <= '힣':  # 한글 음절인지 확인
                code = ord(char) - ord('가')
                initial = code // (21 * 28)   # 초성
                medial = (code % (21 * 28)) // 28   # 중성
                final = code % 28   # 종성
                separated.append({
                    'initial': self.initial_consonants[initial],
                    'medial': self.medial_vowels[medial],
                    'final': self.final_consonants[final] if final != 0 else None
                })
            else:
                separated.append({'initial': char, 'medial': None, 'final': None})
        return separated

    def create_hangul_images(self, word): # 자모음 이미지 생성
        separated = self.split_hangul(word)
        images = []
        
        for char_info in separated:
            width, height = 50, 100
            img = Image.new('RGB', (width, height), color=(255, 255, 255))
            draw = ImageDraw.Draw(img)

            if char_info['initial'] is not None:
                draw.text((10, 30), char_info['initial'], fill=(0, 0, 0))
            if char_info['medial'] is not None:
                draw.text((20, 60), char_info['medial'], fill=(0, 0, 0))
            if char_info['final'] is not None:
                draw.text((30, 90), char_info['final'], fill=(0, 0, 0))

            images.append(img)
        return images


# 실시간 웹캠 
def process_frame(frame):
    if frame is None:
        return None
    
    # PIL 이미지로 변환
    pil_image = Image.fromarray(frame)
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Azure Custom Vision 예측
        results = predictor.detect_image(CUSTOM_VISION_PROJECT_ID, CUSTOM_VISION_PUBLISHED_NAME, img_byte_arr)
        annotated_frame = draw_boxes(frame, results.predictions)
        return annotated_frame
    
    except Exception as e:
        print(f"Error during prediction: {e}")
        return frame

# 이미지에 경계 박스 그리기
def draw_boxes(image, predictions):
    img = image.copy()
    for pred in predictions:
        if pred.probability > 0.9:
            color = (255, 0, 0)
            box = pred.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])
        
            # 경계 박스 그리기
            cv2.rectangle(img, (left, top), (left + width, top + height), color, 2)
            label = f"{pred.tag_name}: {pred.probability:.2f}"
            cv2.putText(img, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return img

# Chatbot 
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={  # Azure Search와 관련된 설정
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()  # 속성 접근 방식으로 수정
        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
        video_url = citations[-2]
        image_urls = citations[-3].split(' ')   # 이미지 URL이 여러 개일 수 있음

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        # speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, chat_history, video_url, image_urls

    except Exception as e:
        return [("Error", str(e))], chat_history


# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=3):
            gr.Markdown("# Azure OpenAI + Cognitive Search + Speech 기반 수화 챗봇")
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
        with gr.Column(scale=2):
            video_display = gr.Video(label="수어 영상")
            image_display = gr.Gallery(label="수어 이미지", columns=3)

    with gr.Row():
        toggle_button = gr.Button("지문자 입력 시작하기")

        # 버튼 클릭으로 표시/숨기기되는 영역
        with gr.Column(visible=False) as customvision_section:
            webcam = gr.Video(label="Webcam Feed")
            webcam_output = gr.Image(label="인식 결과")

        with gr.Column(visible=False) as hangul_section:
            word_image_output = gr.Gallery(label="단어의 자음/모음 이미지")

    # 대화 기록 저장
    visible_state = gr.State(value=False)
    state = gr.State([])

    # 이벤트 연결    
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, state, video_display, image_display])
    clear_button.click(lambda: [], None, chatbot)

    # 버튼 클릭 시 토글 상태 변경
    toggle_button.click(
        lambda visible: (not visible, gr.update(visible=not visible), gr.update(visible=True)),
        inputs=[visible_state],
        outputs=[visible_state, customvision_section, hangul_section],
    )

    # 웹캠 실시간 처리
    webcam.change(process_frame, inputs=webcam, outputs=webcam_output)

    # 텍스트 입력 시 자음/모음 이미지 생성
    user_input.submit(create_hangul_image, [user_input], word_image_output)

demo.launch()


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
